# copilot_ET_comprehensive_ai_usage_analysis

This notebook is an improved, safer, and more reproducible version of "ET_comprehensive_ai_usage_analysis.ipynb".
It includes robust data loading, guarded statistical tests, feature engineering, and an ML pipeline with cross-validation and grid search.

**Author**: Copilot (created for EmanToraih-AI)
**Date**: 2026-01-05


## Requirements

Run this cell to (optionally) install required packages in the notebook environment.
If your environment already provides these packages you can skip installation.


In [ ]:
# Uncomment to install missing packages in some environments
# !pip install -q scikit-learn xgboost lightgbm shap joblib

import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Core imports and configuration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.api.types import is_numeric_dtype

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score)
import joblib

pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 5)

print('Imports ready')


In [ ]:
# Robust data loader and helpers
REQUIRED_COLUMNS = [
    'SessionID', 'SessionDate', 'UsedAgain', 'StudentLevel', 'TaskType',
    'Discipline', 'FinalOutcome', 'SessionLengthMin', 'TotalPrompts',
    'AI_AssistanceLevel', 'SatisfactionRating'
]

def load_data(path='ai_assistant_usage_student_life.csv'):
    df = pd.read_csv(path)
    missing = [c for c in REQUIRED_COLUMNS if c not in df.columns]
    if missing:
        raise ValueError(f'Missing required columns: {missing}')
    # Parse dates safely
    df['SessionDate'] = pd.to_datetime(df['SessionDate'], errors='coerce')
    if df['SessionDate'].isna().any():
        print('Warning: Some SessionDate values could not be parsed and are NaT')
    # Standardize UsedAgain to boolean (support 0/1/'True'/'False')
    df['UsedAgain'] = df['UsedAgain'].map({1: True, 0: False, '1': True, '0': False, 'True': True, 'False': False}).fillna(df['UsedAgain'])
    df['UsedAgain'] = df['UsedAgain'].astype(bool)
    return df

def safe_value_counts_bool(series):
    counts = series.value_counts()
    return int(counts.get(True, 0)), int(counts.get(False, 0))


In [ ]:
# Load dataset
df = load_data('ai_assistant_usage_student_life.csv')
print(f'Dataset shape: {df.shape}')
display(df.head())


In [ ]:
# Safe UsedAgain distribution and plot
true_count, false_count = safe_value_counts_bool(df['UsedAgain'])
total = len(df)
print(f'UsedAgain: True={true_count}, False={false_count}, total={total}')
print(f'Percentages -> True: {true_count/total*100:.2f}%, False: {false_count/total*100:.2f}%')

fig, ax = plt.subplots(1, 2, figsize=(12, 4))
ax[0].bar(['True','False'], [true_count, false_count], color=['#51cf66','#ff6b6b'])
ax[0].set_title('UsedAgain counts')
ax[1].pie([true_count, false_count], labels=[f'True ({true_count/total*100:.1f}%)', f'False ({false_count/total*100:.1f}%)'],
          colors=['#51cf66','#ff6b6b'], startangle=90)
ax[1].set_title('UsedAgain percentage')
plt.tight_layout()
plt.show()


In [ ]:
# Feature engineering (safe)
df_fe = df.copy()
# Efficiency metrics
df_fe['MinPerPrompt'] = df_fe['SessionLengthMin'] / df_fe['TotalPrompts']
df_fe['MinPerPrompt'].replace([np.inf, -np.inf], np.nan, inplace=True)
df_fe['SatisfactionPerPrompt'] = df_fe['SatisfactionRating'] / df_fe['TotalPrompts']
df_fe['SatisfactionPerPrompt'].replace([np.inf, -np.inf], np.nan, inplace=True)
df_fe['PromptsPerMin'] = df_fe['TotalPrompts'] / df_fe['SessionLengthMin']
df_fe['PromptsPerMin'].replace([np.inf, -np.inf], np.nan, inplace=True)
# Binary features
df_fe['HighAssistance'] = (df_fe['AI_AssistanceLevel'] >= 3).astype(int)
df_fe['HighSatisfaction'] = (df_fe['SatisfactionRating'] >= 4).astype(int)
df_fe['LongSession'] = (df_fe['SessionLengthMin'] >= df_fe['SessionLengthMin'].median()).astype(int)
df_fe['SuccessfulOutcome'] = (df_fe['FinalOutcome'] == 'Assignment Completed').astype(int)

display(df_fe[['MinPerPrompt','SatisfactionPerPrompt','PromptsPerMin']].describe())


In [ ]:
# Prepare data for ML using ColumnTransformer + Pipeline
feature_cols = ['StudentLevel','Discipline','TaskType','FinalOutcome',
                'SessionLengthMin','TotalPrompts','AI_AssistanceLevel','SatisfactionRating',
                'MinPerPrompt','SatisfactionPerPrompt','PromptsPerMin',
                'HighAssistance','HighSatisfaction','LongSession','SuccessfulOutcome']
X = df_fe[feature_cols].copy()
y = df_fe['UsedAgain'].astype(int)

categorical_cols = ['StudentLevel','Discipline','TaskType','FinalOutcome']
numeric_cols = [c for c in feature_cols if c not in categorical_cols]  # keep engineered numeric columns

# Column transformer
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_cols),
    ('cat', OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore'), categorical_cols)
], remainder='drop')

pipe = Pipeline([
    ('pre', preprocessor),
    ('clf', RandomForestClassifier(class_weight='balanced', random_state=42, n_jobs=-1))
])

print('Pipeline ready')


In [ ]:
# Cross-validated grid search (example)
param_grid = {
    'clf__n_estimators': [100, 200],
    'clf__max_depth': [None, 10, 20]
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(pipe, param_grid, cv=cv, scoring='roc_auc', n_jobs=-1, verbose=1)

# Fit on the entire dataset (you may prefer to split train/test first)
grid.fit(X, y)
print('Best CV ROC-AUC:', grid.best_score_)
print('Best params:', grid.best_params_)

best_model = grid.best_estimator_

# Evaluate on a held-out test set for a final estimate
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:,1]
print('Test Accuracy:', accuracy_score(y_test, y_pred))
print('Test Precision:', precision_score(y_test, y_pred))
print('Test Recall:', recall_score(y_test, y_pred))
print('Test F1:', f1_score(y_test, y_pred))
print('Test ROC-AUC:', roc_auc_score(y_test, y_proba))


In [ ]:
# Save the trained model to disk
joblib.dump(best_model, 'copilot_best_model.joblib')
print('Saved model to copilot_best_model.joblib')


## Notes and next steps

- This notebook focuses on safety, reproducibility, and a robust ML pipeline.
- Consider adding SHAP explainability for the final tree-based model (install shap).
- Consider handling missing values explicitly (imputation) before modeling for production usage.
- If the dataset is imbalanced, try resampling (SMOTE) or threshold tuning in addition to class_weight.

You can compare this notebook to the original and adapt additional analysis cells as needed.
